In [1]:
import numpy as np                # import numpy
from tqdm import tqdm

#import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as tt

if torch.cuda.is_available():
  print('Running on Graphics')
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
  print('Running on Processor')

Running on Graphics


In [2]:
train_data = CIFAR10('../../CIFAR/cifar_data/', train=True, download=True,transform=tt.Compose(
    [tt.RandomCrop(32, padding=4),
     tt.RandomHorizontalFlip(),
     tt.ToTensor(),
     tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ]))
test_data = CIFAR10('../../CIFAR/cifar_data/', train=False, download=True,transform=tt.Compose(
    [tt.ToTensor(),
     tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
def get_acc(yhat, y):
    predictions = torch.argmax(yhat, dim=1)
    total_correct = torch.sum(predictions == y, dtype=torch.float32)
    return total_correct

In [4]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.c1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.act = nn.ReLU()
        self.bn1 = nn.BatchNorm2d((in_channels))
        self.bn2 = nn.BatchNorm2d((out_channels))
        if in_channels == out_channels:
            self.id = nn.Identity()
        else:
            self.id = nn.Conv2d(in_channels, out_channels, 1)
        
    def forward(self, x):
        x_res = self.c1(self.act(self.bn1(x)))
        x_res = self.c2(self.act(self.bn2(x_res)))
        return x_res + self.id(x)

class bottleneck(nn.Module):
  def __init__(self):
    super().__init__()
    self.b0 = block(32,32)
    self.b1 = block(32,7)
    self.b2 = block(7,32)
    self.b3 = block(32,32)

  def forward(self, x):
    self.bn = self.b1(self.b0(x))
    x = self.b3(self.b2(self.bn))
    return  x


class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.b1 = block(3,16)
    self.b2 = block(16,32)
    self.pool = nn.MaxPool2d(2)

  def forward(self, x):
    x = self.pool(self.b1(x))
    x = self.pool(self.b2(x))
    return x


class Decoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.b1 = block(32,16)
    self.b2 = block(16,3)
    self.u1 = nn.ConvTranspose2d(32,32,2,2)
    self.u2 = nn.ConvTranspose2d(16,16,2,2)
    self.act = nn.Tanh()

  def forward(self, x):
    x = self.b1(self.u1(x))
    x = self.b2(self.u2(x))
    return self.act(x)

class DNA(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.bn1 = bottleneck()
        self.bn2 = bottleneck()
        
    def forward(self, x):
        x = self.enc(x)
        r1 = self.dec(self.bn1(x))
        r2 = self.dec(self.bn2(x))
        return r1, r2

In [5]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = nn.Conv2d(3,64,3,padding=1)
        self.r1 = block(64,128)
        self.r2 = block(128,256)
        self.r3 = block(256, 512)
        self.classifier = nn.Sequential(nn.MaxPool2d(4), nn.Flatten(start_dim=1), nn.Linear(512, 10))
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        
    def forward(self,x):
        x = self.pool(self.act(self.c1(x)))
        x = self.pool(self.r1(x))
        x = self.pool(self.r2(x))
        x = self.r3(x)
        x = self.classifier(x)
        return x

In [6]:
def train(forward_fs, loss_f, optim, train_data, test_data, models, epochs, batch_size, auto=False, eta=0, adv_path=None):
    metrics = []
    for i in tqdm(range(epochs)):
        t_loss=0
        for model in models:
            model.train()
        dl = iter(DataLoader(train_data, batch_size=batch_size, shuffle=True))
        for idx in range(len(dl)):
          x,y = next(dl)
          x = x.to(device)
          y = y.to(device)
          for forward_f in forward_fs:
              if eta != 0:
                y_hat = forward_f(gen_FGSM(x, y, eta, adv_path, models))
              else:
                y_hat = forward_f(x)
              if auto:
                  loss = loss_f(y_hat, x)
              else:
                  loss = loss_f(y_hat, y)
              for model in models:
                  model.zero_grad()
              loss.backward()
              optim.step()
        for model in models:
            model.eval()
        t_loss = 0
        print('TESTING')
        for idx, (x, y) in enumerate(DataLoader(test_data, batch_size=batch_size)):
          x = x.to(device)
          y = y.to(device)
          y_hat = forward_f(x)
          if auto:
              loss = loss_f(y_hat, x)
          else:
              loss = get_acc(y_hat, y)
          t_loss = t_loss + loss.data
        print(t_loss/len(test_data))
        metrics.append(t_loss/len(test_data))
    return np.transpose(metrics)

In [7]:
def get_R(X,Y):
    #First modify to create nonsingular X:
    _,R = torch.linalg.qr(X)
    cols = torch.diag(R)
    cols = abs(cols/torch.max(cols))>0.0005
    X = X[:,cols]

    X = torch.cat([X, torch.ones([batch_size,1]).to(device)],dim=1)
    Yhat = torch.matmul(torch.matmul(X,torch.linalg.pinv(X)),Y)
    Ehat = Y - Yhat
    SSres = torch.sum(torch.square(Ehat))
    Ybar = torch.mean(Y, dim=0).unsqueeze(0)
    SStot = torch.sum(torch.square(Y-Ybar))
    eta = 0.001 #constant for stability
    R = 1 - SSres/(SStot+eta)
    return torch.log(SStot+eta)-torch.log(SSres+eta)

In [23]:
auto_loss = nn.MSELoss()
lambda1 = 0.05
def total_loss(xr, x):
    L1 = auto_loss(x, xr[0])
    L2 = auto_loss(x, xr[1])
    L3 = get_R(torch.flatten(dna.bn1.bn, start_dim=1), torch.flatten(dna.bn2.bn, start_dim=1))
    total_loss = torch.sqrt(L1**2 + L2**2) + lambda1*L3
   # print('MSE1:{} MSE2:{} R2:{}'.format(L1,L2,L3))
    return total_loss

In [24]:
#Train AE
dna = DNA().to(device)
batch_size = 1250
learning_rate = 5.0e-4
epochs = 30

optimizer = optim.Adam(dna.parameters(), lr = learning_rate)

models = [dna]
forward_both = lambda x: dna(x)

metric = train([forward_both], total_loss, optimizer, train_data, test_data, models, epochs, batch_size, auto=True)

  0%|          | 0/30 [00:00<?, ?it/s]

TESTING


  3%|▎         | 1/30 [00:14<06:47, 14.07s/it]

tensor(0.0002, device='cuda:0')
TESTING


  7%|▋         | 2/30 [00:28<06:35, 14.11s/it]

...

tensor(3.0816e-05, device='cuda:0')
TESTING


 97%|█████████▋| 29/30 [06:51<00:14, 14.17s/it]

tensor(3.2025e-05, device='cuda:0')
TESTING


100%|██████████| 30/30 [07:05<00:00, 14.18s/it]

tensor(3.1592e-05, device='cuda:0')


In [25]:
#Train Classifier
cls = Classifier().to(device)

batch_size = 500
learning_rate = 5.0e-4
epochs = 30

optimizer = optim.Adam(list(cls.parameters()), lr = learning_rate)
loss_ce = nn.CrossEntropyLoss()

models = [cls]
forward_c1 = lambda x: cls(dna(x)[0])
forward_c2 = lambda x: cls(dna(x)[1])


metric = train([forward_c1,forward_c2], loss_ce, optimizer, train_data, test_data, models, epochs, batch_size, auto=False)

  0%|          | 0/30 [00:00<?, ?it/s]

TESTING


  3%|▎         | 1/30 [00:11<05:25, 11.23s/it]

tensor(0.4663, device='cuda:0')
TESTING


  7%|▋         | 2/30 [00:23<05:23, 11.57s/it]

...

tensor(0.7652, device='cuda:0')
TESTING


 97%|█████████▋| 29/30 [06:02<00:12, 12.73s/it]

tensor(0.7401, device='cuda:0')
TESTING


100%|██████████| 30/30 [06:15<00:00, 12.52s/it]

tensor(0.7516, device='cuda:0')


In [31]:
torch.save(dna.state_dict(), 'models/dna')
torch.save(cls.state_dict(), 'models/cls_dna')